# Import CSV

In [2]:
import pandas as pd

In [3]:
fileName = 'open-slovenian-championship_full_list_01072021.xlsx'
filePath = '../input/'

data = pd.read_excel(filePath + fileName, sheet_name='Sheet1', engine='openpyxl')


# Create PDFs

In [4]:
from fpdf import FPDF
import datetime

In [5]:
fontSize = 14
fontHeight = 7
titleSize = 20
footerSize = 8
bigSpace = 15
midSpace = 7
smallSpace = 5
emptyPages = 0

In [6]:
title = 'Krvavec Open 2021'
subtitle = 'Application form'
emptyLine = '____________'
meetDirector = 'Klub Letalcev Vrhnika, Jurij Vidic'
responsibility = 'Participation in this competition is at the sole responsibility of each individual pilot. The event organizer does not claim any responsibility for the pilots or other third parties.'

In [7]:
class PDF(FPDF):
    def header(self):
        # Logo
        self.image(filePath + 'logo.png', 10, 8, 33)
        
        # TITLE
        pdf.add_font('Bold', '', '../input/fonts/font_bold.ttf', uni=True)
        pdf.set_font('Bold', '', 8)
        self.set_font('Bold', '', titleSize)
        # Calculate width of title and position
        w = self.get_string_width(title) + 6
        w *= 0.8
        self.set_x((210 - w) / 2)
        self.cell(w, 9, title,'C')
        
        # Line break
        self.ln(10)
        
        # SUBTITLE
        self.set_font('Reg', '', titleSize)
        w = self.get_string_width(subtitle) + 6
        w *= 0.8
        self.set_x((210 - w) / 2)
        self.cell(0, 9,subtitle,'C')
        
        # Line break
        self.ln(30)

    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        pdf.add_font('Reg', '', '../input/fonts/font_reg.ttf', uni=True)
        pdf.set_font('Reg', '', 8)
        self.set_font('Reg', '', footerSize)
        # Text color in gray
        self.set_text_color(128)
        # Page number
        
        self.cell(0, 10, 'Document is automatically generated with FPDF. Author: Mitja Jančič', 0, 0, 'C')

In [8]:
import re

def deEmojify(text):
    regrex_pattern = re.compile(pattern="["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'', text)


In [9]:
def getValueFromData(PDF, Pilot, Property, FriendlyName, FontSize, LineBreak, EmptyLine = '', FontHeight = 7):
    PDF.set_font('Reg', '', FontSize)
    PDF.write(FontHeight, FriendlyName)
    PDF.set_font('Bold', '', FontSize)
    
    if Property == 'NULL':
        entry = EmptyLine
    else:
        entry = deEmojify(str(Pilot[Property]))
        
    if (entry == 'nan'):
        entry = EmptyLine
        
    PDF.write(FontHeight, entry)
    PDF.ln(LineBreak)


In [10]:
pdf = PDF()
pdf.set_left_margin(20)
pdf.set_right_margin(20)
pdf.add_font('Reg', '', '../input/fonts/font_reg.ttf', uni=True)
pdf.add_font('Bold', '', '../input/fonts/font_bold.ttf', uni=True)

for index, pilot in data.iterrows():
    pdf.add_page()
    
    # Pilot name
    getValueFromData(pdf, pilot, 'Name', 'Pilot name: ', fontSize, midSpace)
    getValueFromData(pdf, pilot, 'Surname', 'Pilot surname: ', fontSize, midSpace)
    # Gender
    getValueFromData(pdf, pilot, 'Gender', 'Gender: ', fontSize, midSpace)
    # Licence no.
    getValueFromData(pdf, pilot, 'National federation license id',
                     'Licence no.: ', fontSize, midSpace)
    # FAI Sporting license
    getValueFromData(pdf, pilot, 'Fai license number',
                     'FAI number: ', fontSize, midSpace)
    # CIVL ID
    getValueFromData(pdf, pilot, 'Civl id', 'CIVL ID: ', fontSize, midSpace)

    # Date of birth
    # getValueFromData(pdf, pilot, 'Date of birth',
    #                  'Date of birth: ', fontSize, midSpace)

    # Mobile phone
    getValueFromData(pdf, pilot, 'Mobile tel number',
                     'Mobile phone: ', fontSize, midSpace)
    # team
    getValueFromData(pdf, pilot, 'Club', 'Club: ', fontSize, midSpace, EmptyLine = 2 * emptyLine)
    # sponsor
    getValueFromData(pdf, pilot, 'NULL', 'Sponsors: ', fontSize, midSpace)
    # nationality
    getValueFromData(pdf, pilot, 'Country of residence', 'Nation: ', fontSize, midSpace)
    
    # glider
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontHeight, 'Paraglider (EN 926/2): ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontHeight, str(
        pilot['Paraglider manufacturer']) + ' ' + str(pilot['Paraglider model']))
    pdf.ln(midSpace)
    
    # glider class
    getValueFromData(pdf, pilot, 'Paraglider certification',
                     'Safety class: ', fontSize, midSpace)
    # glider color
    getValueFromData(pdf, pilot, 'Paraglider color',
                     'Paraglider color: ', fontSize, bigSpace, EmptyLine=emptyLine)
    
    # health insurance
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Medical insurance, which includes paragliding competitions:')
    pdf.ln(10)
    getValueFromData(pdf, pilot, 'NULL', 'Company: ', fontSize, midSpace, EmptyLine=2*emptyLine)
    getValueFromData(pdf, pilot, 'NULL', 'Insurance card/policy number: ', fontSize, midSpace)
    
    # Responsibility
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Responsibility:')
    pdf.ln(10)
    pdf.set_font('Reg', '', fontSize)
    pdf.write(7, responsibility)
    pdf.ln(bigSpace)
    
    # Organisor
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Meet director: ')
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, meetDirector)
    pdf.ln(midSpace)
    
    # Signature
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Signature by organizer: ')
    pdf.ln(bigSpace)
    
    # Date created
    pdf.ln(smallSpace)
    pdf.set_font('Reg', '', 10)
    date = datetime.datetime.now().strftime("%I:%M:%S %p on %B %d, %Y.")
    pdf.write(fontSize, 'Application form auto-generated at ' + date)


In [11]:
for i in range(emptyPages):
    pdf.add_page()
    
    # Pilot name
    getValueFromData(pdf, pilot, 'NULL', 'Pilot name: ', fontSize, midSpace,)
    # Gender
    getValueFromData(pdf, pilot, 'NULL', 'Gender: ', fontSize, midSpace)
    # Licence no.
    getValueFromData(pdf, pilot, 'NULL', 'Licence no.: ', fontSize, midSpace)
    # FAI Sporting license
    getValueFromData(pdf, pilot, 'NULL', 'FAI number: ', fontSize, midSpace)
    # CIVL ID
    getValueFromData(pdf, pilot, 'NULL', 'CIVL ID: ', fontSize, midSpace)
    # Date of birth
    getValueFromData(pdf, pilot, 'NULL', 'Date of birth: ', fontSize, midSpace)
    # Mobile phone
    getValueFromData(pdf, pilot, 'NULL', 'Mobile phone: ', fontSize, midSpace)    
    # flying since
    getValueFromData(pdf, pilot, 'NULL', 'Flying since: ', fontSize, midSpace)
    # team
    getValueFromData(pdf, pilot, 'NULL', 'Team: ', fontSize, midSpace)
    # team
    getValueFromData(pdf, pilot, 'NULL', 'Sponsors: ', fontSize, midSpace)
    # nationality
    getValueFromData(pdf, pilot, 'NULL', 'Nation: ', fontSize, midSpace)
    
    # glider
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontHeight, 'Paraglider (EN 926/2): ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontHeight, '')
    pdf.ln(midSpace)
    
    # glider class
    getValueFromData(pdf, pilot, 'NULL', 'Safety class: ', fontSize, midSpace)
    # glider color
    getValueFromData(pdf, pilot, 'NULL', 'Paraglider color: ', fontSize, bigSpace)
    
    # health insurance
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Medical insurance, which includes paragliding competitions:')
    pdf.ln(10)
    getValueFromData(pdf, pilot, 'NULL', 'Company: ', fontSize, midSpace)
    getValueFromData(pdf, pilot, 'NULL', 'Card/policy number: ', fontSize, bigSpace)
    
    # Responsibility
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Responsibility:')
    pdf.ln(10)
    pdf.set_font('Reg', '', fontSize)
    pdf.write(7, responsibility)
    pdf.ln(bigSpace)
    
    # Organisor
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Meet director: ')
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, meetDirector)
    pdf.ln(midSpace)
    
    # Signature
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Signature by organizer: ')
    pdf.ln(bigSpace)
    
    # Date created
    pdf.ln(smallSpace)
    pdf.set_font('Reg', '', 10)
    date = datetime.datetime.now().strftime("%I:%M:%S %p on %B %d, %Y.")
    pdf.write(fontSize, 'Application form auto-generated at ' + date)

In [12]:
pdf.output('../output/ApplicationForms.pdf', 'F')

''